# Model Report

In [1]:
import warnings
from pathlib import Path
from time import time

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotnine as gg
import pymc3 as pm
import seaborn as sns
from src.command_line_interfaces import simulation_based_calibration_cli as sbc_cli
from src.context_managers import set_directory
from src.data_processing import common as dphelp
from src.modeling import pymc3_analysis as pmanal
from src.modeling import pymc3_sampling_api as pmapi
from src.modeling import sampling_pymc3_models as sampling
from src.modeling.sampling_pymc3_models import SamplingArguments
from src.modeling.simulation_based_calibration_helpers import SBCFileManager
from src.plot.color_pal import SeabornColor

notebook_tic = time()

warnings.simplefilter(action="ignore", category=UserWarning)

gg.theme_set(gg.theme_classic())
%config InlineBackend.figure_format = "retina"

RANDOM_SEED = 847
np.random.seed(RANDOM_SEED)

pymc3_cache_dir = Path("..", "models", "modeling_cache", "pymc3_model_cache")

Parameters for papermill:

- `MODEL`: which model was tested
- `SBC_RESULTS_DIR`: directory containing results of many rounds of SBC
- `NUM_SIMULATIONS`: the number of simiulations; will be used to check that all results are found

## Setup

### Papermill parameters

In [2]:
MODEL = ""
SBC_RESULTS_DIR = ""
NUM_SIMULATIONS = -1

In [3]:
# Parameters
MODEL = "crc_ceres_mimic_one"
SBC_RESULTS_DIR = "temp/crc_ceres_mimic_one"
NUM_SIMULATIONS = 5

### Prepare and validate papermill parameters

Build the model using the `MODEL` parameter.

In [4]:
ModelClass = sbc_cli.get_model_class(sbc_cli.ModelOption[MODEL])

Check values passed as the directory with results of the rounds of SBC.

In [5]:
sbc_results_dir = Path("../..", SBC_RESULTS_DIR)
assert sbc_results_dir.is_dir()
assert sbc_results_dir.exists()

Confirm that there is a positive number of simulations.

In [6]:
assert NUM_SIMULATIONS > 0

## Read in all results

In [7]:
for sbc_dir in sbc_results_dir.iterdir():
    sbc_fm = SBCFileManager(sbc_dir)
    if not sbc_fm.all_data_exists():
        raise Exception(f"Not all output from '{sbc_fm.dir.name}' exist.")
    res = sbc_fm.get_sbc_results()

In [8]:
res.posterior_summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
parameter,,,,,,,,,
μ_h,0.404,0.338,-0.198,1.046,0.010,0.007,1102.0,877.0,NaN
μ_d,0.112,0.118,-0.112,0.329,0.004,0.003,875.0,981.0,NaN
μ_η,0.133,0.205,-0.285,0.482,0.007,0.005,990.0,1013.0,NaN
h[0],0.750,0.691,-0.406,2.187,0.022,0.016,989.0,934.0,NaN
h[1],0.250,0.714,-1.015,1.602,0.023,0.017,972.0,983.0,NaN
...,...,...,...,...,...,...,...,...,...
μ[146],0.964,0.881,-0.618,2.598,0.028,0.020,966.0,992.0,NaN
μ[147],1.583,0.920,-0.241,3.234,0.028,0.020,1096.0,980.0,NaN
μ[148],1.578,0.923,-0.028,3.520,0.029,0.021,1002.0,941.0,NaN


---

In [9]:
notebook_toc = time()
print(f"execution time: {(notebook_toc - notebook_tic) / 60:.2f} minutes")

execution time: 0.03 minutes


In [10]:
%load_ext watermark
%watermark -d -u -v -iv -b -h -m

Last updated: 2021-04-01

Python implementation: CPython
Python version       : 3.9.2
IPython version      : 7.21.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 3.10.0-1062.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit

Hostname: compute-a-16-163.o2.rc.hms.harvard.edu

Git branch: simulation-based-calibration

pymc3     : 3.11.1
arviz     : 0.11.2
plotnine  : 0.7.1
matplotlib: 3.3.4
numpy     : 1.20.1
pandas    : 1.2.3
seaborn   : 0.11.1

